# InceptionV3

In [34]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import imageio

# Load the pre-trained InceptionV3 model and modify the final layer
model = models.inception_v3(pretrained=False)
num_classes = 13  # Replace with the number of classes in your checkpoint
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("best_model_inception.pth"))
model.eval()  # Set the model to evaluation mode


# Function to register hooks to capture feature maps for all layers
def register_hooks(model):
    activation = {}
    hooks = []

    def get_activation(layer_name):
        def hook(model, input, output):
            activation[layer_name] = output

        return hook

    for name, layer in model.named_modules():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            hooks.append(layer.register_forward_hook(get_activation(name)))
    return activation, hooks


# Register hooks for all layers
activation, hooks = register_hooks(model)


# Function to preprocess the input image
def preprocess_image(img_path):
    preprocess = transforms.Compose(
        [
            transforms.Resize((299, 299)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )
    img = Image.open(img_path).convert("RGB")
    img_tensor = preprocess(img).unsqueeze(0)  # Add batch dimension
    return img_tensor


# Function to compute and save average activation maps
def save_average_activation_maps(model, activation, results_dir, img_size=(299, 299)):
    os.makedirs(results_dir, exist_ok=True)

    for idx, layer_name in enumerate(activation.keys()):
        print(f"Computing average activation for layer {layer_name}")
        feature_maps = activation[layer_name]

        # Compute the average activation map
        if feature_maps.dim() == 4:  # For Conv2d layers
            avg_activation_map = (
                torch.mean(feature_maps, dim=1).squeeze().detach().cpu().numpy()
            )
        elif feature_maps.dim() == 2:  # For Linear layers
            avg_activation_map = feature_maps.squeeze().detach().cpu().numpy()
            avg_activation_map = avg_activation_map.reshape((1, -1))

        avg_activation_map = (avg_activation_map - avg_activation_map.min()) / (
            avg_activation_map.max() - avg_activation_map.min()
        )

        # Ensure the activation map is 2D
        if avg_activation_map.ndim == 3:
            avg_activation_map = avg_activation_map[0]

        # Save the average activation map
        plt.imsave(
            os.path.join(results_dir, f"{idx:03d}_{layer_name}_avg_activation.png"),
            avg_activation_map,
            cmap="viridis",
        )


# Function to create a GIF from images
def create_gif(image_folder, output_path, duration=0.5):
    images = []
    for file_name in sorted(os.listdir(image_folder)):
        if file_name.endswith(".png"):
            file_path = os.path.join(image_folder, file_name)
            img = Image.open(file_path).convert("RGB")
            img = img.resize((299, 299), Image.LANCZOS)  # Resize to the same dimensions
            images.append(np.array(img))

    imageio.mimsave(output_path, images, duration=duration)


# List of image paths
image_paths = [
    "Posters//1979//tt0080180//tt0080180.jpg",
    "Posters//1981//tt0081256//tt0081256.jpg",
    "Posters//1992//tt0104389//tt0104389.jpg",
    "Posters//1988//tt0098415//tt0098415.jpg",
]

# Loop through each image, process it, and save the activations in separate folders
for img_path in image_paths:
    img_name = os.path.basename(img_path).split(".")[0]
    input_tensor = preprocess_image(img_path)
    input_tensor.requires_grad_(True)  # Ensure the input tensor requires gradients

    # Forward pass to capture activations
    model(input_tensor)

    # Directory to save the average activation maps for the current image
    results_dir = f"average_activations_inceptionv3_{img_name}"

    # Save average activation maps for the current image
    save_average_activation_maps(model, activation, results_dir)

    # Create a GIF from the saved activation maps
    gif_output_path = f"average_activations_inceptionv3_{img_name}.gif"
    create_gif(results_dir, gif_output_path)

    print(
        f"Visualization and GIF creation complete for {img_path}. Check the '{results_dir}' and '{gif_output_path}' for results."
    )

print("All visualizations and GIFs complete.")

Computing average activation for layer Conv2d_1a_3x3.conv
Computing average activation for layer Conv2d_2a_3x3.conv
Computing average activation for layer Conv2d_2b_3x3.conv
Computing average activation for layer Conv2d_3b_1x1.conv
Computing average activation for layer Conv2d_4a_3x3.conv
Computing average activation for layer Mixed_5b.branch1x1.conv
Computing average activation for layer Mixed_5b.branch5x5_1.conv
Computing average activation for layer Mixed_5b.branch5x5_2.conv
Computing average activation for layer Mixed_5b.branch3x3dbl_1.conv
Computing average activation for layer Mixed_5b.branch3x3dbl_2.conv
Computing average activation for layer Mixed_5b.branch3x3dbl_3.conv
Computing average activation for layer Mixed_5b.branch_pool.conv
Computing average activation for layer Mixed_5c.branch1x1.conv
Computing average activation for layer Mixed_5c.branch5x5_1.conv
Computing average activation for layer Mixed_5c.branch5x5_2.conv
Computing average activation for layer Mixed_5c.branch3

# Resnet

In [33]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import imageio

# Load the pre-trained ResNet model and modify the final layer
model = models.resnet50(pretrained=False)
num_classes = 13  # Replace with the number of classes in your checkpoint
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("best_model_resnet50.pth"))
model.eval()  # Set the model to evaluation mode


# Function to register hooks to capture feature maps for all layers
def register_hooks(model):
    activation = {}
    hooks = []

    def get_activation(layer_name):
        def hook(model, input, output):
            activation[layer_name] = output

        return hook

    for name, layer in model.named_modules():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            hooks.append(layer.register_forward_hook(get_activation(name)))
    return activation, hooks


# Register hooks for all layers
activation, hooks = register_hooks(model)


# Function to preprocess the input image
def preprocess_image(img_path):
    preprocess = transforms.Compose(
        [
            transforms.Resize((299, 299)),  # Resize to 299x299 to match InceptionV3
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )
    img = Image.open(img_path).convert("RGB")
    img_tensor = preprocess(img).unsqueeze(0)  # Add batch dimension
    return img_tensor


# Function to compute and save average activation maps
def save_average_activation_maps(model, activation, results_dir, img_size=(299, 299)):
    os.makedirs(results_dir, exist_ok=True)

    for idx, layer_name in enumerate(activation.keys()):
        print(f"Computing average activation for layer {layer_name}")
        feature_maps = activation[layer_name]

        # Compute the average activation map
        if feature_maps.dim() == 4:  # For Conv2d layers
            avg_activation_map = (
                torch.mean(feature_maps, dim=1).squeeze().detach().cpu().numpy()
            )
        elif feature_maps.dim() == 2:  # For Linear layers
            avg_activation_map = feature_maps.squeeze().detach().cpu().numpy()
            avg_activation_map = avg_activation_map.reshape((1, -1))

        avg_activation_map = (avg_activation_map - avg_activation_map.min()) / (
            avg_activation_map.max() - avg_activation_map.min()
        )

        # Ensure the activation map is 2D
        if avg_activation_map.ndim == 3:
            avg_activation_map = avg_activation_map[0]

        # Save the average activation map
        plt.imsave(
            os.path.join(results_dir, f"{idx:03d}_{layer_name}_avg_activation.png"),
            avg_activation_map,
            cmap="viridis",
        )


# Function to create a GIF from images
def create_gif(image_folder, output_path, duration=0.5):
    images = []
    for file_name in sorted(os.listdir(image_folder)):
        if file_name.endswith(".png"):
            file_path = os.path.join(image_folder, file_name)
            img = Image.open(file_path).convert("RGB")
            img = img.resize((299, 299), Image.LANCZOS)  # Resize to the same dimensions
            images.append(np.array(img))

    imageio.mimsave(output_path, images, duration=duration)


# List of image paths
image_paths = [
    "Posters//1979//tt0080180//tt0080180.jpg",
    "Posters//1981//tt0081256//tt0081256.jpg",
    "Posters//1992//tt0104389//tt0104389.jpg",
    "Posters//1988//tt0098415//tt0098415.jpg",
]

# Loop through each image, process it, and save the activations in separate folders
for img_path in image_paths:
    img_name = os.path.basename(img_path).split(".")[0]
    input_tensor = preprocess_image(img_path)
    input_tensor.requires_grad_(True)  # Ensure the input tensor requires gradients

    # Forward pass to capture activations
    model(input_tensor)

    # Directory to save the average activation maps for the current image
    results_dir = f"average_activations_resnet_{img_name}"

    # Save average activation maps for the current image
    save_average_activation_maps(model, activation, results_dir)

    # Create a GIF from the saved activation maps
    gif_output_path = f"average_activations_resnet_{img_name}.gif"
    create_gif(results_dir, gif_output_path)

    print(
        f"Visualization and GIF creation complete for {img_path}. Check the '{results_dir}' and '{gif_output_path}' for results."
    )

print("All visualizations and GIFs complete.")

Computing average activation for layer conv1
Computing average activation for layer layer1.0.conv1
Computing average activation for layer layer1.0.conv2
Computing average activation for layer layer1.0.conv3
Computing average activation for layer layer1.0.downsample.0
Computing average activation for layer layer1.1.conv1
Computing average activation for layer layer1.1.conv2
Computing average activation for layer layer1.1.conv3
Computing average activation for layer layer1.2.conv1
Computing average activation for layer layer1.2.conv2
Computing average activation for layer layer1.2.conv3
Computing average activation for layer layer2.0.conv1
Computing average activation for layer layer2.0.conv2
Computing average activation for layer layer2.0.conv3
Computing average activation for layer layer2.0.downsample.0
Computing average activation for layer layer2.1.conv1
Computing average activation for layer layer2.1.conv2
Computing average activation for layer layer2.1.conv3
Computing average activ